In [1]:
import warnings
warnings.filterwarnings('ignore')
import json

## Get API Keys

In [2]:
from utils import load_env
load_env()

In [3]:
from utils import get_tavily_api_key
TAVILY_API_KEY = get_tavily_api_key()

## LLama3.1 Model Test

In [4]:
from utils import llama31

In [5]:
messages = [
  {"role": "user",
    "content": "Who wrote the book Charlotte's Web?"}
]

In [6]:
response = llama31(messages)
print(response)

The book "Charlotte's Web" was written by E.B. White.


## brave_search buit-in tool

In [7]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

23 January 2025


In [8]:
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [9]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Menlo Park, California?"}
  ]

In [10]:
response = llama31(messages)
print(response)

brave_search.call(query="current weather in Menlo Park, California")


brave_seach in the built-in search tool for Llama. We need Tavily to do the search. 

### Calling the search API

In [11]:
from utils import cprint
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

result = tavily_client.search("current weather in Menlo Park, California")
cprint(result)

{
    "query": "current weather in Menlo Park, California",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Menlo Park, California",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1737663895, 'localtime': '2025-01-23 12:24'}, 'current': {'last_updated_epoch': 1737663300, 'last_updated': '2025-01-23 12:15', 'temp_c': 16.2, 'temp_f': 61.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 32, 'wind_dir': 'NNE', 'pressure_mb': 1028.0, 'pressure_in': 30.35, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 36, 'cloud': 0, 'feelslike_c': 16.2, 'feelslike_f': 61.2, 'windchill_c': 15.5, 'wind

In [12]:
search_result = result["results"][0]["content"]
print(search_result)

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1737663895, 'localtime': '2025-01-23 12:24'}, 'current': {'last_updated_epoch': 1737663300, 'last_updated': '2025-01-23 12:15', 'temp_c': 16.2, 'temp_f': 61.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 32, 'wind_dir': 'NNE', 'pressure_mb': 1028.0, 'pressure_in': 30.35, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 36, 'cloud': 0, 'feelslike_c': 16.2, 'feelslike_f': 61.2, 'windchill_c': 15.5, 'windchill_f': 59.9, 'heatindex_c': 15.8, 'heatindex_f': 60.4, 'dewpoint_c': 3.5, 'dewpoint_f': 38.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 2.7, 'gust_mph': 2.3, 'gust_kph': 3.7}}


### Promting Llama with search tool response

In [13]:
messages = [
    {
        "role": "system",
        "content": system_prompt_content,
    },
    {
        "role": "user", 
       "content": "What is the current weather in Menlo Park, California?",
    },
    {
        "role":"assistant",
        "content": response,
    },
    {
        "role": "ipython",
        "content": search_result,
    },
]

In [14]:
response = llama31(messages)
print(response)

The current weather in Menlo Park, California is sunny with a temperature of 16.2°C (61.2°F) and a humidity of 36%. The wind is blowing at a speed of 3.6 km/h (2.2 mph) from the north-northeast direction.


## Code interpreter buit-in tool

In [15]:
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [16]:
loan_question = (
    "How much is the monthly payment, total payment, "
    "and total interest paid for a 30 year mortgage of $1M "
    "at a fixed rate of 6% with a 20% down payment?"
)

In [17]:
messages = [
    {"role": "system",     "content": system_prompt_content},
    {"role": "user",       "content": loan_question},
  ]

In [18]:
response = llama31(messages)
print(response)

import math

# Define the variables
loan_amount = 1000000  # $1M
interest_rate = 0.06  # 6%
loan_term = 30  # 30 years
down_payment = 200000  # 20% of $1M

# Calculate the loan amount after down payment
loan_amount_after_down_payment = loan_amount - down_payment

# Calculate the monthly interest rate
monthly_interest_rate = interest_rate / 12

# Calculate the number of payments
number_of_payments = loan_term * 12

# Calculate the monthly payment
monthly_payment = loan_amount_after_down_payment * (monthly_interest_rate * (1 + monthly_interest_rate) ** number_of_payments) / ((1 + monthly_interest_rate) ** number_of_payments - 1)

# Calculate the total payment
total_payment = monthly_payment * number_of_payments

# Calculate the total interest paid
total_interest_paid = total_payment - loan_amount_after_down_payment

print(f"Monthly payment: ${monthly_payment:.2f}")
print(f"Total payment: ${total_payment:.2f}")
print(f"Total interest paid: ${total_interest_paid:.2f}")


The output of the language model is a python code. We run the code and return the tools output back to the LLM. 

In [19]:
from utils import calculate_loan
monthly_payment, total_payment, total_interest_paid = calculate_loan(
    loan_amount = 1000000, 
    annual_interest_rate = 0.06, 
    loan_term = 30, 
    down_payment = 200000)

print(f"Monthly payment: ${(monthly_payment)}")
print(f"Total payment: ${(total_payment)}")
print(f"Total interest paid: ${(total_interest_paid)}")

Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705


### Reprompting Llama with code interpreter response

In [20]:
code_interpreter_tool_response="""
Monthly payment: $4796
Total payment: $1726705
Total interest paid: $926705
"""

In [21]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",      "content": loan_question},
    {"role": "assistant", "content": response},
    {"role": "ipython",   "content": code_interpreter_tool_response}
  ]

In [22]:
response = llama31(messages)
print(response)

The monthly payment for a 30-year mortgage of $1M at a fixed rate of 6% with a 20% down payment is $4,796. The total payment over the life of the loan is $1,726,705, and the total interest paid is $926,705.


## Custom tool calling

In [36]:
from utils import trending_songs, get_boiling_point

country = "US"
top_num = 5
top_songs = trending_songs(country, top_num)
print(f"Top {top_num} trending songs in {country}:")
print(top_songs)

Top 5 trending songs in US:
['Blinding Lights - The Weeknd', 'Levitating - Dua Lipa', 'Peaches - Justin Bieber', 'Save Your Tears - The Weeknd', 'Good 4 U - Olivia Rodrigo']


In [55]:
system_prompt_content = f"""
Environment: ipython
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [56]:
user_prompt = """
Answer the user's question by using the following functions if needed.
If none of the functions can be used, please say so.
Functions (in JSON format):
{
    "type": "function", "function": {
        "name": "get_boiling_point",
        "description": "Get the boiling point of a liquid",
        "parameters": {
            "type": "object", "properties": [
                {"liquid_name": {"type": "object", "description": "name of the liquid"}},
                {"celsius": {"type": "object", "description": "whether to use celsius"}}
            ], "required": ["liquid_name"]
        }
    }
}
{
    "type": "function", "function": {
        "name": "trending_songs",
        "description": "Returns the trending songs on a Music site",
        "parameters": {
            "type": "object", "properties": [
                {"country": {"type": "object", "description": "country to return trending songs for"}},
                {"n": {"type": "object", "description": "The number of songs to return"}}
            ], "required": ["country"]
        }
    }
}

Question: Can you check the top 5 trending songs in US and list then in the output?
"""

In [57]:
messages = [
    {
      "role": "system", 
      "content": system_prompt_content
    },
    {
        "role": "user", 
        "content": user_prompt
    }
  ]

In [58]:
response = llama31(messages, 405)
print(response)

{"type": "function", "name": "trending_songs", "parameters": {"country": "US", "n": "5"}}


### Calling the custom tool 

In [59]:
custom_tools = {"trending_songs": trending_songs,
                "get_boiling_point": get_boiling_point}

In [60]:
res = json.loads(response)
function_name = res['name']
parameters = list(res['parameters'].values())
function_name, parameters

('trending_songs', ['US', '5'])

In [61]:
tool_result = custom_tools[function_name](*parameters)
tool_result

['Blinding Lights - The Weeknd',
 'Levitating - Dua Lipa',
 'Peaches - Justin Bieber',
 'Save Your Tears - The Weeknd',
 'Good 4 U - Olivia Rodrigo']

### Reprompting Llama with custom tool call result

In [62]:
messages = [
    {
      "role": "system", 
      "content": system_prompt_content
    },
    {
        "role": "user", 
        "content": user_prompt
    },
    {
        "role": "assistant",
        "content": response
    },
    {
        "role": "ipython",
        "content": ",".join(tool_result)
    }
  ]

In [63]:
response = llama31(messages, 70)
print(response)

The top 5 trending songs in the US are:

1. Blinding Lights - The Weeknd
2. Levitating - Dua Lipa
3. Peaches - Justin Bieber
4. Save Your Tears - The Weeknd
5. Good 4 U - Olivia Rodrigo
